In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
558,tt0179626,60000000,56359980,15 Minutes,Robert De Niro|Charlize Theron|Edward Burns|Ke...,John Herzfeld,America Likes to Watch,When Eastern European criminals Oleg and Emil ...,120,Action|Crime|Thriller,New Line Cinema|Industry Entertainment|Tribeca...,3/1/2001,5.7,2001
1557,tt0433442,15000000,11992014,The Return,Adam Scott|Kate Beahan|Sarah Michelle Gellar|P...,Asif Kapadia,The past never dies. It kills.,Joanna Mills has a successful career but feels...,85,Drama|Thriller,Rogue Pictures|Intrepid Pictures|Biscayne Pict...,11/10/2006,5.2,2006
1478,tt0455967,18000000,41009669,John Tucker Must Die,Jesse Metcalfe|Brittany Snow|Sophia Bush|Ariel...,Betty Thomas,Don't Get Mad. Get Even.,After discovering they are all dating the same...,87,Comedy,Twentieth Century Fox Film Corporation|Dune En...,7/27/2006,6.0,2006
1848,tt0146882,30000000,47126295,High Fidelity,John Cusack|Iben Hjejle|Todd Louiso|Jack Black...,Stephen Frears,"A comedy about fear of commitment, hating your...",When record store owner Rob Gordon gets dumped...,113,Comedy|Drama|Romance|Music,Buena Vista|Touchstone Pictures,3/17/2000,7.0,2000
1505,tt0466909,25000000,119188334,The Omen,Liev Schreiber|Julia Stiles|Seamus Davey-Fitzp...,John Moore,A new generation of terror,A diplomatic couple adopts the son of the devi...,110,Drama|Horror|Thriller,Twentieth Century Fox Film Corporation|11:11 M...,6/6/2006,5.2,2006


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# создание столбца с номерами месяцев, в которых выходили фильмы
data['month'] = data['release_date']
data['month'] = data['month'].apply(lambda x: x.split('/')[0])
data['month'] = pd.to_numeric(data['month'])

# создание столбца с количеством символов в названии фильмов
data['title_len'] = data.original_title.apply(lambda x: len(x))

# создание столбца с количеством слов в описании фильмов
data['words_in_overview'] = data.overview.apply(lambda x: len(x.split()))

# создание столбца с прибылью
data['profit'] = data.revenue - data.budget

In [6]:
# раскодирование столбца с названиями фильмов, чтобы русские символы стали читаемы
data.original_title=data.original_title.apply(lambda x: x.encode('cp1252').decode('utf8'))

# перевод столбца с датой из string формата в datetime
data.release_date = pd.to_datetime(data['release_date'])

In [7]:
# 
def genre_counts(movie_genres, counter):
    """
    функция для подсчёта жанров в датасете (задания 11, 12, 17)
    параметры:
    movie_genres - значение текущей ячейки из столбца genres
    counter - счётчик (класс Counter из collections)
    """
    movie_genres = movie_genres.split("|")
    for genre in movie_genres:
        counter[genre] += 1

In [8]:
def data_count(value, column, counter, data=data, count=False, summary=False, sum_column=''):
    """
    функция считает суммарные значения столбца sum_column 
    или подсчитывает количество повторяющихся значений value в предоставленном датафрейме
    параметры:
    value - значение текущей ячейки
    column - название столбца, по которому отфильровываются все повторяющиеся value
    counter - счётчик (класс Counter из collections)
    data - весь датафрейм для обработки. стандартно равен всему датафрейму
    count - нужно ли в счётчике вести подсчёт повторяющихся значений. 
    summary - нужно ли в счётчике суммировать значения. значения суммируются в столбце sum_column
    sum_column - столбец с численными значениями,которые нужно суммировать 
    """
    # old_values - список уже обработанных значений. если новое значение не входит в список, оно обрабатывается
    if value not in old_values:
        old_values.append(value)
        # выбор всех одинаковых значений в таблице для последующей обработки
        filtered_data = data[data[column] == value]
        # если в отфильтрованной таблице есть хотя бы 1 строка, подсчитываем данные
        if filtered_data.shape[0] > 0:
            total = 0
            if count is True:
                total = filtered_data.shape[0]
            elif summary is True:
                total = filtered_data[sum_column].sum()
            if "|" in value:
                values = value.split("|")
                for value in values:
                    counter[value] += total
            else:
                counter[value] += total

In [9]:
# ф
def actors_crossing(actors, counter):
    """
    функция для подсчёта задания 27 - какие актёры чаще всего играют в одном фильме
    параметры:
    actors - значение текущей ячейки из столбца cast
    counter - счётчик (класс Counter из collections)
    """
    if "|" in actors:
        actors = actors.split("|")
        
        # сортировка актёров по алфавиту, чтобы одинаковые актёры всегда встречались в одном порядке 
        # в разных фильмах
        actors.sort()
        for i in range(len(actors)):
            for y in range(i+1, len(actors)):
                crossed_actors = actors[i] + " & " + actors[y]
                counter[crossed_actors] += 1

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [10]:
# ответ
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'
# +

In [11]:
max_budg = data.budget.max()
display(data[data.budget == max_budg].original_title)

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

ВАРИАНТ 2

In [12]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [13]:
# ответ
answers['2'] = 'Gods and Generals'
# +

In [14]:
max_runtime = data.runtime.max()
display(data[data.runtime == max_runtime].original_title)

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [15]:
answers['3'] = 'Winnie the Pooh'
# +

In [16]:
min_runtime = data.runtime.min()
display(data[data.runtime == min_runtime].original_title)

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [17]:
answers['4'] = '110'
# +

In [18]:
display(round(data.runtime.mean()))

110

# 5. Каково медианное значение длительности фильмов? 

In [19]:
answers['5'] = '107'
# +

In [20]:
display(data.runtime.median())

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [21]:
answers['6'] = 'Avatar'
# +

In [22]:
max_profit = data.profit.max()
display(data[data.profit == max_profit].original_title)

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [23]:
answers['7'] = 'The Lone Ranger'
# +

In [24]:
max_loss = data.profit.min()
display(data[data.profit == max_loss].original_title)

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
answers['8'] = '1478'
# +

In [26]:
display(data.profit[data.profit > 0].count())

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [27]:
answers['9'] = 'The Dark Knight'
# +

In [28]:
max_profit_2008 = data.profit[data.release_year == 2008].max()
display(data[data.profit == max_profit_2008].original_title)

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [29]:
answers['10'] = 'The Lone Ranger'
# +

In [30]:
max_loss_2012_2014 = data.profit[data.release_year.between(2012, 2013, 2014)].min()
display(data[data.profit == max_loss_2012_2014].original_title)

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [31]:
answers['11'] = 'Drama'
# +

In [32]:
genre_counter = Counter()
popular_genres = data.genres.apply(lambda x: genre_counts(x, genre_counter))
display(genre_counter.most_common(1))

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
answers['12'] = 'Drama'
# +

In [34]:
%%time
profit_genres = Counter()

# фильтр прибыльных фильмов 
profit_movies = data[data.profit > 0]

popular_genres = profit_movies.genres.apply(lambda x: genre_counts(x, profit_genres))
print(profit_genres.most_common(1))

[('Drama', 560)]
CPU times: user 19.3 ms, sys: 661 µs, total: 20 ms
Wall time: 22.3 ms


ВТОРОЙ ВАРИАНТ

In [35]:
%%time
# второй вариант (намного более медленный)
profit_genres = Counter()
for i in range(data.shape[0]):
    # берём только значения прибыльных фильмов (по индексу ячейки)
    if int(data.iloc[[i], [-1]].values[0][0]) > 0:
        # обновляем значения счётчика
        genre_counts(data.iloc[[i], [9]].values[0][0], profit_genres)
print(profit_genres.most_common(1))

[('Drama', 560)]
CPU times: user 8.6 s, sys: 19.1 ms, total: 8.61 s
Wall time: 8.61 s


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [36]:
answers['13'] = 'Peter Jackson'
# +

In [37]:
%%time
revenue_by_director = Counter()

# создание списка уже использованных значений (используется в функции data_count)
old_values = []

data.director.apply(lambda x: data_count(x, 'director', counter=revenue_by_director, 
                                         summary=True, sum_column='revenue'))
print(revenue_by_director.most_common(1))

[('Peter Jackson', 6490593685)]
CPU times: user 2.55 s, sys: 0 ns, total: 2.55 s
Wall time: 2.57 s


ВТОРОЙ ВАРИАНТ

In [38]:
%%time
revenue_by_director = Counter()
directors = data.director.unique()
for director in directors:
    director_movies = data[data.director == director]
    # раздление списка режиссёров у фильма на несколько (разные режиссёры разделены "|")
    director = director.split("|")
    # добавление каждого из режиссёров в счётчик
    for sub_director in director:
        revenue_by_director[sub_director] += director_movies.revenue.sum()
print(revenue_by_director.most_common(1))

[('Peter Jackson', 6490593685)]
CPU times: user 2.59 s, sys: 0 ns, total: 2.59 s
Wall time: 2.59 s


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [39]:
answers['14'] = 'Robert Rodriguez'
# +

In [40]:
action_by_directors = Counter()
# создание списка уже использованных значений (используется в функции data_count)
old_values = []

# фильтрация фильмов жанра Экшн
action_movies = data[data.genres.str.contains('Action')]

action_movies.director.apply(lambda x: data_count(x, 'director', counter=action_by_directors, data=action_movies,
                                         count=True))
print(action_by_directors.most_common(1))

[('Robert Rodriguez', 9)]


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [41]:
answers['15'] = 'Chris Hemsworth'
# +

In [42]:
actor_revenue = Counter()

# создание списка уже использованных значений (используется в функции data_count)
old_values = []

# фильтрация фильмов, выпущенных в 2012
movies_2012 = data[data.release_year == 2012]

movies_2012.cast.apply(lambda x: data_count(x, 'cast', counter=actor_revenue, 
                                            summary=True, sum_column='revenue', data=movies_2012))
print(actor_revenue.most_common(1))

[('Chris Hemsworth', 2027450773)]


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
answers['16'] = 'Matt Damon'
# +

In [44]:
high_budget_counter = Counter()
old_values=[]

# фильтрация высокобюджетных фильмов - их бюджет выше среднего 
high_budget_movies = data[data.budget > data.budget.mean()]

high_budget_movies.cast.apply(lambda x: data_count(x, 'cast', counter=high_budget_counter,
                                                   count = True, data=high_budget_movies))
high_budget_counter.most_common(1)

[('Matt Damon', 18)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [45]:
answers['17'] = 'Action'
# +

In [46]:
cage_genres = Counter()

# фильтрация фильмов с участием Николаса Кейджа
cage_movies = data[data.cast.str.contains('Nicolas Cage')]

cage_movies.genres.apply(lambda x: genre_counts(x, cage_genres))
cage_genres.most_common(1)

[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [47]:
answers['18'] = 'K-19: The Widowmaker'
# +

In [48]:
# фильтрация фильмов от Paramount Pictures
paramount_movies = data[data.production_companies.str.contains('Paramount Pictures')]
display(paramount_movies[paramount_movies.profit == paramount_movies.profit.min()].original_title)

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [49]:
answers['19'] = '2015'
# +

In [50]:
# группировка фильмов по году выпуска
year_groups = data.groupby('release_year')[['revenue']].sum()
year_groups[year_groups.revenue == year_groups.revenue.max()]

,revenue
release_year,
2015,25449202382


# 20. Какой самый прибыльный год для студии Warner Bros?

In [51]:
answers['20'] = '2014'
# +

In [52]:
# фильтрация фильмов от Warner Bros.
warner_movies = data[data.production_companies.str.contains('Warner Bros')]

# группировка фильмов по году выпуска
year_groups = warner_movies.groupby('release_year')[['profit']].sum()
year_groups[year_groups.profit == year_groups.profit.max()]

,profit
release_year,
2014,2295464519


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
answers['21'] = 'Сентябрь'
# +

In [54]:
# группировка фильмов по месяцу выпуска
month_groups = data.groupby('month')[['imdb_id']].count()

month_groups[month_groups.imdb_id == month_groups.imdb_id.max()]

,imdb_id
month,
9,227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [55]:
answers['22'] = '450'
# +

In [56]:
summer_count = data[data.month.between(6, 8)].count()
summer_count.imdb_id

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [57]:
answers['23'] = 'Peter Jackson'
# +

In [58]:
# зимние месяцы - 1,2 и 12 в году
winter_movies = data[(data.month < 3) | (data.month > 11)]
winter_movies.director.mode()

0    Peter Jackson
dtype: object

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [59]:
answers['24'] = 'Four By Two Productions'
# +

In [60]:
# подсчёт количества символов в названии всех фильмов в разрезе каждой студии
long_titles = Counter()
old_values = []
data.production_companies.apply(lambda x: data_count(x, 'production_companies', counter=long_titles, 
                                                     sum_column='title_len', summary = True, data=data))

# подсчёт количества фильмов у каждой студии
companies = Counter()
old_values = []
data.production_companies.apply(lambda x: data_count(x, 'production_companies', counter=companies, 
                                                     count = True, data=data))

# подсчёт среднего количества символов в названии фильма у каждой студии
avg_long_titles = Counter()
# убеждаемся, что количество компаний в обоих счётчиках совпадает
if len(companies.keys()) == len(long_titles):
    for company in companies.keys():
        avg_long_titles[company] = long_titles[company] / companies[company]
avg_long_titles.most_common(1)

[('Four By Two Productions', 83.0)]

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
answers['25'] = 'Universal Pictures'
# +

In [62]:
# подсчёт количества слов в описании всех фильмов в разрезе каждой студии
long_overviews = Counter()
old_values = []
data.production_companies.apply(lambda x: data_count(x, 'production_companies', counter=long_overviews, data=data, 
                                                     sum_column='words_in_overview', summary = True))

# подсчёт среднего количества символов в названии фильма у каждой студии 
avg_long_overviews = Counter()
# убеждаемся, что количество компаний в обоих счётчиках совпадает
if len(companies.keys()) == len(long_titles):
    for company in companies.keys():
        avg_long_overviews[company] = long_overviews[company] / companies[company]
avg_long_overviews.most_common(1)

[('Midnight Picture Show', 175.0)]

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [63]:
answers['26'] = 'Inside Out', 'The Dark Knight', '12 Years a Slave'
# +

In [64]:
one_percent = round(data.shape[0]/100)
best_rating = data.nlargest(one_percent, 'vote_average')
display(best_rating.original_title)

599                                   The Dark Knight
9                                          Inside Out
34                                               Room
118                                      Interstellar
125                                The Imitation Game
119                           Guardians of the Galaxy
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
28                                          Spotlight
124                                        Big Hero 6
155                          The Theory of Everything
177                            The Fault in Our Stars
283                         

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [65]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

In [66]:
actors_cross_counter = Counter()
data.cast.apply(lambda x: actors_crossing(x, actors_cross_counter))
actors_cross_counter.most_common(5)

[('Daniel Radcliffe & Emma Watson', 8),
 ('Daniel Radcliffe & Rupert Grint', 8),
 ('Emma Watson & Rupert Grint', 8),
 ('Ben Stiller & Owen Wilson', 6),
 ('Helena Bonham Carter & Johnny Depp', 6)]

# Submission

In [67]:
# ответы на каждый вопрос
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '110',
 '5': '107',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': '1478',
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Universal Pictures',
 '26': ('Inside Out', 'The Dark Knight', '12 Years a Slave'),
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [68]:
# убеждаемся, что ответов 27 штук
len(answers)

27